In [ ]:
import tensorflow as tf

load_img = tf.keras.preprocessing.image.load_img
img_to_array = tf.keras.preprocessing.image.img_to_array
array_to_img = tf.keras.preprocessing.image.array_to_img
Adam = tf.keras.optimizers.Adam
fashion_mnist = tf.keras.datasets.fashion_mnist
to_categorical = tf.keras.utils.to_categorical
ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator
ModelCheckpoint = tf.keras.callbacks.ModelCheckpoint
load_model = tf.keras.models.load_model

from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objs as go
from plotly import subplots
import plotly

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), 'Physical GPUs,', len(logical_gpus), 'Logical GPUs')
    except RuntimeError as e:
        print(e)

In [ ]:
cat = load_img('cat.jpg')
cat

In [ ]:
cat = img_to_array(cat)
print(cat.shape)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(3, # number of filter layers
                (3,                 # y dimension of kernel
                 3),                # x dimension of kernel
                 input_shape=cat.shape))

model.summary()

In [ ]:
cat_batch = np.expand_dims(cat,axis=0)
cat_batch.shape

In [ ]:
conv_cat = model(cat_batch)
conv_cat.shape

In [ ]:
conv_cat = np.squeeze(conv_cat, axis=0)
conv_cat.shape

In [ ]:
conv_cat = array_to_img(conv_cat)
display(conv_cat)

In [ ]:
def visualize_cat(model, cat):
    cat_batch = np.expand_dims(cat,axis=0)
    conv_cat = model(cat_batch)
    conv_cat = np.squeeze(conv_cat, axis=0)
    conv_cat = array_to_img(conv_cat)
    display(conv_cat)

In [ ]:
visualize_cat(model, cat)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(3,(10, 10), input_shape=cat.shape))

model.summary()

In [ ]:
visualize_cat(model, cat)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(1,(3,3),input_shape=cat.shape))

model.summary()

In [ ]:
def visualize_cat_one_channel(model, cat):
    cat_batch = np.expand_dims(cat,axis=0)
    conv_cat2 = model(cat_batch)
    conv_cat2 = np.squeeze(conv_cat2, axis=0)
    conv_cat2 = array_to_img(conv_cat2)
    display(conv_cat2)

In [ ]:
visualize_cat_one_channel(model, cat)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(1,(20,20),input_shape=cat.shape))

model.summary()

In [ ]:
visualize_cat_one_channel(model, cat)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(1,(20,20),input_shape=cat.shape))
model.add(tf.keras.layers.Activation('relu'))

model.summary()

In [ ]:
visualize_cat_one_channel(model, cat)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(1,(3,3),input_shape=cat.shape))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(5,5)))

model.summary()

In [ ]:
visualize_cat_one_channel(model, cat)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(1,(3,3),input_shape=cat.shape))

model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(5,5)))
model.summary()

In [ ]:
visualize_cat_one_channel(model, cat)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(1,(3,3),input_shape=cat.shape))

model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(3,3)))

model.add(tf.keras.layers.Conv2D(1,(3,3)))

model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(3,3)))

model.summary()

In [ ]:
visualize_cat_one_channel(model, cat)

# **Fashion-MNIST Model**

In [ ]:
IMG_ROWS = 28
IMG_COLS = 28
NUM_CLASSES = 10
VAL_SIZE = 0.2
RANDOM_STATE = 99

BATCH_SIZE = 128

In [ ]:
(train_data, y), (test_data, y_test) = fashion_mnist.load_data()

print("Fashion MNIST train -  rows:",train_data.shape[0]," columns:", train_data.shape[1], " rows:", train_data.shape[2])
print("Fashion MNIST test -  rows:",test_data.shape[0]," columns:", test_data.shape[1], " rows:", train_data.shape[2])

In [ ]:
for i in range(9):
    plt.subplot(330 + 1 + i)    
    plt.imshow(train_data[i], cmap=plt.get_cmap('gray'))

plt.savefig('fashion_mnist.jpeg', dpi=300)

In [ ]:
print(train_data.shape, test_data.shape)

train_data = train_data.reshape((train_data.shape[0], 28, 28, 1))
test_data = test_data.reshape((test_data.shape[0], 28, 28, 1))

print(train_data.shape, test_data.shape)

In [ ]:
train_data = train_data / 255.0
test_data = test_data / 255.0

In [ ]:
y = to_categorical(y)
y_test = to_categorical(y_test)

print(y.shape, y_test.shape)
y[:10]

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(train_data, y, test_size=VAL_SIZE, random_state=RANDOM_STATE)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

In [ ]:
def create_trace(x,y,ylabel,color):
        trace = go.Scatter(
            x = x,y = y,
            name=ylabel,
            marker=dict(color=color),
            mode = "markers+lines",
            text=x
        )
        return trace
    
def plot_accuracy_and_loss(train_model):
    hist = train_model.history
    acc = hist['accuracy']
    val_acc = hist['val_accuracy']
    loss = hist['loss']
    val_loss = hist['val_loss']
    epochs = list(range(1,len(acc)+1))
    
    trace_ta = create_trace(epochs,acc,"Training accuracy", "Green")
    trace_va = create_trace(epochs,val_acc,"Validation accuracy", "Red")
    trace_tl = create_trace(epochs,loss,"Training loss", "Blue")
    trace_vl = create_trace(epochs,val_loss,"Validation loss", "Magenta")
    fig = subplots.make_subplots(rows=1,cols=2, subplot_titles=('Training and validation accuracy',
                                                             'Training and validation loss'))
    fig.append_trace(trace_ta,1,1)
    fig.append_trace(trace_va,1,1)
    fig.append_trace(trace_tl,1,2)
    fig.append_trace(trace_vl,1,2)
    fig['layout']['xaxis'].update(title = 'Epoch')
    fig['layout']['xaxis2'].update(title = 'Epoch')
    fig['layout']['yaxis'].update(title = 'Accuracy', range=[0,1])
    fig['layout']['yaxis2'].update(title = 'Loss', range=[0,1])

    plotly.offline.iplot(fig, filename='accuracy-loss')

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=0.05,    #Randomly rotate images in the range
        zoom_range = 0.2,       # Randomly zoom image
        width_shift_range=0.1,  #Randomly shift images horizontally
        height_shift_range=0.1, #Randomly shift images vertically
        shear_range=0.05        #Randomly shear images
)

datagen.fit(x_train)

In [ ]:
model = tf.keras.Sequential()

#1. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same', input_shape=(28, 28, 1), name = 'conv2d1'))
model.add(tf.keras.layers.BatchNormalization(name = 'batch_norm1'))
model.add(tf.keras.layers.Activation("relu", name = 'relu1'))
model.add(tf.keras.layers.Dropout(0.3, name = 'dropout1'))

#2. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same', name = 'conv2d2'))
model.add(tf.keras.layers.BatchNormalization(name = 'batch_norm2'))
model.add(tf.keras.layers.Activation("relu", name = 'relu2'))

model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), name = 'maxpool2d1'))
model.add(tf.keras.layers.Dropout(0.3, name = 'dropout2'))

#3. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', name = 'conv2d3'))
model.add(tf.keras.layers.BatchNormalization(name = 'batch_norm3'))
model.add(tf.keras.layers.Activation("relu", name = 'relu3'))
model.add(tf.keras.layers.Dropout(0.3, name = 'dropout3'))

#4. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', name = 'conv2d4'))
model.add(tf.keras.layers.BatchNormalization(name = 'batch_norm4'))
model.add(tf.keras.layers.Activation("relu", name = 'relu4'))

model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), name = 'maxpool2d2'))
model.add(tf.keras.layers.Dropout(0.3, name = 'dropout4'))


#FULLY CONNECTED LAYER
model.add(tf.keras.layers.Flatten(name = 'flatten1'))
model.add(tf.keras.layers.Dense(256, name = 'dense1'))
model.add(tf.keras.layers.BatchNormalization(name = 'batch_norm5'))
model.add(tf.keras.layers.Activation("relu", name = 'relu5'))
model.add(tf.keras.layers.Dropout(0.30, name = 'dropout5'))

#OUTPUT LAYER
model.add(tf.keras.layers.Dense(10, activation='softmax', name = 'dense2'))

In [ ]:
optimizer = Adam()
model.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
filepath="weights.best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
NO_EPOCHS = 30
history = model.fit(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                              shuffle=True,
                              epochs=NO_EPOCHS, validation_data = (x_val, y_val),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // BATCH_SIZE,
                              callbacks=callbacks_list)

In [ ]:
plot_accuracy_and_loss(history)

In [ ]:
predict_model = load_model(filepath)

In [ ]:
score = predict_model.evaluate(test_data, y_test,verbose=0)
print("Test Loss:",score[0])
print("Test Accuracy:",score[1])

# **Visualizing Kernel**

In [ ]:
predict_model.summary()

In [ ]:
kernel_bias = []
for layer in predict_model.layers:
    if 'conv' not in layer.name:
        continue
    kernels, biases = layer.get_weights()
    kernel_bias.append([kernels, biases])
    print(layer.name, kernels.shape)

In [ ]:
kernels, biases = kernel_bias[0]
kernels.shape, biases.shape

In [ ]:
k_min, k_max = kernels.min(), kernels.max()
kernels = (kernels - k_min) / (k_max - k_min)

In [ ]:
n_kernels, ix = 32, 1
for i in range(n_kernels):
    f = kernels[:, :, :, i]
    for j in range(1):
        ax = plt.subplot(8, 4, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        plt.imshow(f[:, :, j], cmap='gray')
        ix += 1
plt.savefig('filter.jpeg', dpi=300)

# **Visualizing Feature Map**

In [ ]:
model = tf.keras.models.Model(inputs=predict_model.inputs, outputs=predict_model.layers[0].output)
model.summary()

In [ ]:
img = x_train[10:11]

In [ ]:
feature_maps = model(img)
feature_maps.shape

In [ ]:
ix = 1
for _ in range(8):
    for _ in range(4):
        ax = plt.subplot(8, 4, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        plt.imshow(feature_maps[0, :, :, ix-1], cmap='gray')
        ix += 1
plt.savefig('feature_map1.jpeg', dpi=300)

In [ ]:
model = tf.keras.models.Model(inputs=predict_model.inputs, outputs=predict_model.layers[2].output)
model.summary()

In [ ]:
feature_maps = model(img)
feature_maps.shape

In [ ]:
ix = 1
for _ in range(8):
    for _ in range(4):
        ax = plt.subplot(8, 4, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        plt.imshow(feature_maps[0, :, :, ix-1], cmap='gray')
        ix += 1
plt.savefig('feature_map2.jpeg', dpi=300)

In [ ]:
model = tf.keras.models.Model(inputs=predict_model.inputs, outputs=predict_model.layers[4].output)
model.summary()

In [ ]:
feature_maps = model(img)
feature_maps.shape

In [ ]:
ix = 1
for _ in range(8):
    for _ in range(4):
        ax = plt.subplot(8, 4, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        plt.imshow(feature_maps[0, :, :, ix-1], cmap='gray')
        ix += 1
plt.savefig('feature_map3.jpeg', dpi=300)

In [ ]:
model = tf.keras.models.Model(inputs=predict_model.inputs, outputs=predict_model.layers[16].output)
model.summary()

In [ ]:
feature_maps = model(img)
feature_maps.shape

In [ ]:
ix = 1
for _ in range(8):
    for _ in range(8):
        ax = plt.subplot(8, 8, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        plt.imshow(feature_maps[0, :, :, ix-1], cmap='gray')
        ix += 1
plt.savefig('feature_map4.jpeg', dpi=300)